In [35]:
#Load Libraries
import pandas as pd
from pandas.plotting import scatter_matrix
import pandas_ta as ta
import requests
import json
import numpy as np
import matplotlib.pyplot as plt
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

In [50]:
url='https://www.alphavantage.co/query?function=CRYPTO_INTRADAY&symbol=ETH&market=USD&interval=5min&apikey=D5OBBCW29ZBREHXC'
data = requests.get(url)\
        .json()['Time Series Crypto (5min)']

df = pd.DataFrame.from_dict(data)
#data.splice(index, remove_count )
#dataset = pd.read_json(data, orient ='index')
print(df)



          2022-07-16 20:40:00 2022-07-16 20:35:00 2022-07-16 20:30:00  \
1. open            1330.60000          1327.40000          1320.99000   
2. high            1337.39000          1334.01000          1329.43000   
3. low             1329.72000          1326.51000          1320.99000   
4. close           1334.12000          1330.61000          1327.40000   
5. volume                4302                5153                4880   

          2022-07-16 20:25:00 2022-07-16 20:20:00 2022-07-16 20:15:00  \
1. open            1318.84000          1325.34000          1329.41000   
2. high            1321.00000          1325.36000          1329.42000   
3. low             1315.25000          1314.60000          1320.70000   
4. close           1320.99000          1318.85000          1325.35000   
5. volume                4319               10621                7457   

          2022-07-16 20:10:00 2022-07-16 20:05:00 2022-07-16 20:00:00  \
1. open            1334.87000          1339.24000

In [17]:
#check for data with zero shares traded
index_Zeros = df[ df['Shares Traded'] == '-'].index

dataset.drop(index_Zeros , inplace = True)
dataset.loc[(dataset['Shares Traded'] == '-')]
dataset.isna().sum()

Date                 0
Open                 0
High                 0
Low                  0
Close                0
Shares Traded        0
Turnover (Rs. Cr)    0
dtype: int64

In [24]:
#dataset.ta.indicators() -- list of indicators in ta
#help(ta.atr) -- help
dataset['ATR']

Help on function atr in module pandas_ta.volatility.atr:

atr(high, low, close, length=None, mamode=None, talib=None, drift=None, offset=None, **kwargs)
    Average True Range (ATR)
    
    Averge True Range is used to measure volatility, especially volatility caused by
    gaps or limit moves.
    
    Sources:
        https://www.tradingview.com/wiki/Average_True_Range_(ATR)
    
    Calculation:
        Default Inputs:
            length=14, drift=1, percent=False
        EMA = Exponential Moving Average
        SMA = Simple Moving Average
        WMA = Weighted Moving Average
        RMA = WildeR's Moving Average
        TR = True Range
    
        tr = TR(high, low, close, drift)
        if 'ema':
            ATR = EMA(tr, length)
        elif 'sma':
            ATR = SMA(tr, length)
        elif 'wma':
            ATR = WMA(tr, length)
        else:
            ATR = RMA(tr, length)
    
        if percent:
            ATR *= 100 / close
    
    Args:
        high (pd.Series):